<a href="https://colab.research.google.com/github/shreya139/Circuits-in-Stocks/blob/202218061-JATAN/DataPrepration_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Required Library

In [1]:
!pip install yfinance
!pip install fredapi
!pip install yfinance pandas numpy scikit-learn
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
ext = '0.4.0-oneiric1_amd64.deb -qO'
!wget $url/libta-lib0_$ext libta.deb
!wget $url/ta-lib0-dev_$ext ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.1) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

     ━━━━━━━━

In [4]:
pip install nsepy

  Preparing metadata (setup.py) ... done
  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36056 sha256=edae09d4c364c8bf152b1eae0af48f913b89404b09b92ad72329706d67bd3590
  Stored in directory: /root/.cache/pip/wheels/4c/87/cb/acaf83f625e5fc73e1fe6e2a8e97680c74cd72391850ef5a86
Successfully built nsepy


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from talib import RSI, BBANDS, MACD, EMA, STOCH, ADX
from nsepy import get_history
from datetime import date
import yfinance as yf

In [6]:
import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr

In [33]:
yf.pdr_override() # <== that's all it takes :-)


#this is reference start date before EMA 200 days will fetch null
ref_startdate = date(2008,1,1)

#actual start date and end date from where the Nifty model to be built
startdate = date(2009,1,1)
enddate = date(2022,12,31)


# download dataframe
data = pdr.get_data_yahoo("^NSEI", start=ref_startdate, end=enddate,interval='1d')
data

[*********************100%%**********************]  1 of 1 completed


Open          High           Low         Close  \
Date                                                                 
2008-01-01   6136.750000   6165.350098   6109.850098   6144.350098   
2008-01-02   6144.700195   6197.000000   6060.850098   6179.399902   
2008-01-03   6184.250000   6230.149902   6126.399902   6178.549805   
2008-01-04   6179.100098   6300.049805   6179.100098   6274.299805   
2008-01-07   6271.000000   6289.799805   6193.350098   6279.100098   
...                  ...           ...           ...           ...   
2022-12-26  17830.400391  18084.099609  17774.250000  18014.599609   
2022-12-27  18089.800781  18149.250000  17967.449219  18132.300781   
2022-12-28  18084.750000  18173.099609  18068.349609  18122.500000   
2022-12-29  18045.699219  18229.699219  17992.800781  18191.000000   
2022-12-30  18259.099609  18265.250000  18080.300781  18105.300781   

               Adj Close  Volume  
Date                              
2008-01-01   6144.350098       0  
2008-01-02   6179.399902       0  
2008-01-03   6178.549805       0  
2008-01-04   6274.299805       0  
2008-01-07   6279.100098       0  
...                  ...     ...  
2022-12-26  18014.599609  176700  
2022-12-27  18132.300781  214300  
2022-12-28  18122.500000  193900  
2022-12-29  18191.000000  281100  
2022-12-30  18105.300781  192000  

[3675 rows x 6 columns]

After retrieving Nifty records, many brokerage firms utilize the TALIB API to compute key technical indicators like Exponential Moving Averages (EMA), Moving Average Convergence Divergence (MACD), Relative Strength Index (RSI -14), and Bollinger Bands values. These calculated indicators serve as the foundation for generating visually informative technical charts of the Nifty index. This practice is commonplace among various brokerage firms and is instrumental in creating insightful and actionable trading charts for Nifty.

In [34]:
#calculating technical indicators such as EMA-200, EMA-100, EMA-50, EMA-21, EMA-5, MACD, RSI-14, Bollinger Bands
EMA_200 = EMA(data['Close'],200)
EMA_100 = EMA(data['Close'],100)
EMA_50 = EMA(data['Close'],50)
EMA_21 = EMA(data['Close'],21)
EMA_5 = EMA(data['Close'],5)
data['EMA-200']=EMA_200
data['EMA-100']=EMA_100
data['EMA-50']=EMA_50
data['EMA-21']=EMA_21
data['EMA-5']=EMA_5
data['MACD'], data['macdEMA'], data['macdHistory'] = MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['RSI-14']=RSI(data['Close'],timeperiod=14)

In [35]:
print(data.head(4))

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2008-01-01  6136.750000  6165.350098  6109.850098  6144.350098  6144.350098   
2008-01-02  6144.700195  6197.000000  6060.850098  6179.399902  6179.399902   
2008-01-03  6184.250000  6230.149902  6126.399902  6178.549805  6178.549805   
2008-01-04  6179.100098  6300.049805  6179.100098  6274.299805  6274.299805   

            Volume  EMA-200  EMA-100  EMA-50  EMA-21  EMA-5  MACD  macdEMA  \
Date                                                                         
2008-01-01       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-02       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-03       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-04       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   

            macdHistory  RSI-14  
Date                  

In [36]:
print(data.tail(4))

                    Open          High           Low         Close  \
Date                                                                 
2022-12-27  18089.800781  18149.250000  17967.449219  18132.300781   
2022-12-28  18084.750000  18173.099609  18068.349609  18122.500000   
2022-12-29  18045.699219  18229.699219  17992.800781  18191.000000   
2022-12-30  18259.099609  18265.250000  18080.300781  18105.300781   

               Adj Close  Volume       EMA-200       EMA-100        EMA-50  \
Date                                                                         
2022-12-27  18132.300781  214300  17435.654416  17840.502471  18166.168781   
2022-12-28  18122.500000  193900  17442.488700  17846.086581  18164.456280   
2022-12-29  18191.000000  281100  17449.936574  17852.916550  18165.497210   
2022-12-30  18105.300781  192000  17456.457611  17857.914257  18163.136566   

                  EMA-21         EMA-5       MACD    macdEMA  macdHistory  \
Date                             

Retrieve historical data for significant variables such as

USD/INR

Brent Crude Price,

London Stock Exchange values (FTSE index),

USA stock exchange prices (Dow index),
  
Hong Kong stock exchange (Hang Seng Index), and

the India Volatility Index (VIX) from www.Investing.com .




Also, collect historical data related to

Foreign Institutional Investors (FII),

specifically focusing on FII future index values and

FII option index values.

In [85]:
import yfinance as yf
import pandas as pd
import pytz  # Import the pytz library for timezone handling

# Define the date range for historical data
start_date = "2009-01-01"
end_date = "2022-12-31"

# Create a function to fetch daily historical data for a given ticker symbol
def fetch_daily_data(symbol):
    ticker = yf.Ticker(symbol)
    data = ticker.history(period="1d", start=start_date, end=end_date)
    data = data[['Close']].rename(columns={'Close': symbol})
    return data

# List of ticker symbols for the variables
ticker_symbols = ['USDINR=X', 'BZ=F', '^FTSE', '^DJI', '^HSI']

# Fetch daily historical data for each variable
dataframes = [fetch_daily_data(symbol) for symbol in ticker_symbols]

# Merge all data into one DataFrame based on the date index
combined_data = pd.concat(dataframes, axis=1)

# Set the timezone of the datetime index to Indian Standard Time (IST)
combined_data.index = combined_data.index.tz_localize('UTC').tz_convert('Asia/Kolkata')

# Print the combined data
print(combined_data)

TypeError: ignored

In [75]:
# Create a ticker object for USD/INR
ticker = yf.Ticker("USDINR=X")

# Get historical data for the exchange rate
exchange_rate_data = ticker.history(start=startdate, end=enddate,interval='1d')

# Reset the index and convert it into a column
# exchange_rate_data.reset_index(inplace=True)

# Print the historical exchange rate data
exchange_rate_data[['Close']]

Close
Date                                
2009-01-01 00:00:00+00:00  48.064999
2009-01-02 00:00:00+00:00  48.244999
2009-01-05 00:00:00+00:00  48.229000
2009-01-06 00:00:00+00:00  48.505001
2009-01-07 00:00:00+00:00  48.550999
...                              ...
2022-12-26 00:00:00+00:00  82.604103
2022-12-27 00:00:00+00:00  82.902901
2022-12-28 00:00:00+00:00  82.776100
2022-12-29 00:00:00+00:00  82.851997
2022-12-30 00:00:00+00:00  82.835098

[3646 rows x 1 columns]

### Crude Oil data

In [81]:
ticker = yf.Ticker("BZ=F")

# Get historical data for the exchange rate
df2 = ticker.history(start=startdate, end=enddate,interval='1d')

# Print the historical exchange rate data
df2[['Close']]

Close
Date                                
2009-01-02 00:00:00-05:00  46.910000
2009-01-05 00:00:00-05:00  49.619999
2009-01-06 00:00:00-05:00  50.529999
2009-01-07 00:00:00-05:00  45.860001
2009-01-08 00:00:00-05:00  44.669998
...                              ...
2022-12-23 00:00:00-05:00  83.919998
2022-12-27 00:00:00-05:00  84.330002
2022-12-28 00:00:00-05:00  83.260002
2022-12-29 00:00:00-05:00  82.260002
2022-12-30 00:00:00-05:00  85.910004

[3467 rows x 1 columns]

### df3 - London Stock Exchange values (FTSE index)

In [82]:
ticker = yf.Ticker("^FTSE")

# Get historical data for the exchange rate
df3 = ticker.history(start=startdate, end=enddate,interval='1d')

# Print the historical exchange rate data
df3[['Close']]

Close
Date                                  
2009-01-02 00:00:00+00:00  4561.799805
2009-01-05 00:00:00+00:00  4579.600098
2009-01-06 00:00:00+00:00  4638.899902
2009-01-07 00:00:00+00:00  4507.500000
2009-01-08 00:00:00+00:00  4505.399902
...                                ...
2022-12-22 00:00:00+00:00  7469.299805
2022-12-23 00:00:00+00:00  7473.000000
2022-12-28 00:00:00+00:00  7497.200195
2022-12-29 00:00:00+00:00  7512.700195
2022-12-30 00:00:00+00:00  7451.700195

[3534 rows x 1 columns]

In [76]:
# exchange_rate_data['date_'] = pd.to_datetime(exchange_rate_data.index)

In [77]:
# exchange_rate_data['date_']= exchange_rate_data['date_'].dt.date

In [78]:
# data.index = (pd.to_datetime(data.index))
data['date'] = (pd.to_datetime(data.index))
exchange_rate_data['date'] = (pd.to_datetime(exchange_rate_data.index))
# data['date']= data['date'].dt.date
# exchange_rate_data['date']= exchange_rate_data['date'].dt.date

In [79]:
# data['date'] = pd.to_datetime(data['date'])
# exchange_rate_data['date'] = pd.to_datetime(exchange_rate_data['date'])

In [80]:
# data.index = (pd.to_datetime(data.index))

data= data[startdate:]
# Convert column name Date from String datatype to Date datatype"""

#Sort the Sheet based on Date ascending order. This is one of pre-requisite to merge different dataframe.
#Dataframe that has to be merged should be sorted by join columns. In this case, join column is date.
#Hence, it is sorted based on Date column
# exchange_rate_data=exchange_rate_data.sort_values(by=['date'],ascending=True)
data=pd.merge_asof(data,exchange_rate_data,on='date')
print(data.tail(2))

MergeError: ignored